# AKSSF Topography
### Code to construct DEM for hydroprocessing from 5m DEM available for download
## **Using Timm's 10m composite DEM at this time**

### Set environments, variables, and import modules

In [2]:
# # import modules
# import arcpy
# import os
#
# #Import TauDEM toolbox
# #arcpy.ImportToolbox(r"C:\Program Files\TauDEM\TauDEM5Arc\TauDEM Tools.tbx")
#
# # Set Variables
# drive = "C:\\"
# root_folder = "\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\hydrography"
# # Define data folder
# data_folder = os.path.join(drive, root_folder, 'data\\topography\\USGS3DEP_5m_Alaska\\Bristol_Bay')
# # Set arcpy working environment
# arcpy.env.workspace = "C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\AKSSF_Hydrography.gdb"
# # Variables
# # CSV table from national map viewer
# input_table = "C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\hydrography\\Data\\Raster_CSV_Files\\BB_ifsar_test.csv"
# # Column name of url download link
# url_column = 'col14'
# # Location to store raster downloads
# directory = "C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\hydrography\\Data\\BB_ifsar_dl_test"
# # Define input datasets for merge
# tile_folder = data_folder
# projected_folder = os.path.join(data_folder, 'tiles_projected')
# # Raster of study area with processing regions set by HUC8s
# snap_raster = "C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\hydrography\\AKSSF_Bristol_Bay_StudyArea.tif"
# # Define output raster
# usgs5m_composite = os.path.join(data_folder, 'BBay_Elevation_USGS3DEP_5m_Alaska_AKALB.tif')
# # Define stream network to burn into raster
# flowlines = "D:\\Basedata\\NHD_H_Alaska_State_GDB.gdb\\Hydrography\\NHDFlowline"
# region = "BBay"

## Define Geoprocessing Wrapper

In [1]:
# Arcpy Geoprocessing Wrapper
# Author: Timm Nawrocki
# Last Updated: 2019-10-29
# Usage: Must be executed in an ArcGIS Pro Python 3.6 installation.
# Description: "Arcpy Geoprocessing Wrapper" is a function that wraps other arcpy functions for standardization, input and output checks, and error reporting.
# ---------------------------------------------------------------------------

# Define a wrapper function for arcpy geoprocessing tasks
def arcpy_geoprocessing(geoprocessing_function, check_output = True, check_input = True, **kwargs):
    """
    Description: wraps arcpy geoprocessing and data access functions for file checks, message reporting, and errors.
    Inputs: geoprocessing function -- any arcpy geoprocessing or data access processing steps defined as a function that receive ** kwargs arguments.
            check_output -- boolean input to control if the function should check if the output already exists prior to executing geoprocessing function
            check_input -- boolean input to control if the function should check if the inputs already exist prior to executing geoprocessing function
            **kwargs -- key word arguments that are used in the wrapper and passed to the geoprocessing function
                'input_array' -- if check_input == True, then the input datasets must be passed as an array
                'output_array' -- if check_output == True, then the output datasets must be passed as an array
    Returned Value: Function returns messages, warnings, and errors from geoprocessing functions.
    Preconditions: geoprocessing function and kwargs must be defined, this function does not conduct any geoprocessing on its own
    """

    # Import packages
    import arcpy

    try:
        # If check_output is True, then check if output exists and warn of overwrite if it does
        if check_output == True:
            for output_data in kwargs['output_array']:
                if arcpy.Exists(output_data) == True:
                    print(f"{output_data} already exists and will be overwritten.")
        # If check_input is True, then check if inputs exist and quit if any do not
        if check_input == True:
            for input_data in kwargs['input_array']:
                if arcpy.Exists(input_data) != True:
                    print(f'{input_data} does not exist. Check that environment workspace is correct.')
                    quit()
        # Execute geoprocessing function if all input data exists
        out_process = geoprocessing_function(**kwargs)
        # Provide results report
        try:
            msg_count = out_process.messageCount
            print(out_process.getMessage(msg_count - 1))
        except:
            print(out_process)
    # Provide arcpy errors for execution error
    except arcpy.ExecuteError as err:
        print(arcpy.GetMessages())
        quit()

## Define Merge function

In [3]:
# Merge Source Elevation Tiles
# Author: Timm Nawrocki
# Last Updated: 2019-12-03
# Usage: Must be executed in an ArcGIS Pro Python 3.6 installation.
# Description: "Merge Source Elevation Tiles" is a function that creates single merged DEM from input tiles of the same source with new projection, snap raster, and cell size.
# ---------------------------------------------------------------------------
#
#Define function to merge source elevation tiles
def merge_elevation_tiles(**kwargs):
    """
    Description: creates a DEM from individual DEM tiles
    Inputs: 'tile_folder' -- a folder containing the raster tiles
            'projected_folder' -- a folder in which to store the projected tiles
            'cell_size' -- a cell size for the output DEM
            'input_projection' -- the machine number for the input projection
            'output_projection' -- the machine number for the output projection
            'geographic_transformation -- the string representation of the appropriate geographic transformation (blank if none required)
            'input_array' -- an array containing the snap raster
            'output_array' -- an array containing the output raster
    """

    # Import packages
    import arcpy
    from arcpy.sa import Int
    from arcpy.sa import Raster
    from arcpy.sa import SetNull
    import datetime
    import os
    import time

    # Set overwrite option
    arcpy.env.overwriteOutput = True

    # Use two thirds of cores on processes that can be split.
    arcpy.env.parallelProcessingFactor = "66%"

    # Parse key word argument inputs
    tile_folder = kwargs['tile_folder']
    projected_folder = kwargs['projected_folder']
    workspace = kwargs['workspace']
    cell_size = kwargs['cell_size']
    input_projection = kwargs['input_projection']
    output_projection = kwargs['output_projection']
    geographic_transformation = kwargs['geographic_transformation']
    snap_raster = kwargs['input_array'][0]
    dem_composite = kwargs['output_array'][0]

    # Define intermediate datasets
    mosaic_location, mosaic_name = os.path.split(dem_composite)

    # Start timing function
    iteration_start = time.time()
    # Create a list of DEM raster tiles
    print('Compiling list of raster tiles...')
    arcpy.env.workspace = tile_folder
    tile_list = arcpy.ListRasters('*', 'ALL')
    # Add file path to raster list
    tile_rasters = []
    for tile in tile_list:
        tile_path = os.path.join(tile_folder, tile)
        tile_rasters.append(tile_path)
    # Set environment workspace
    arcpy.env.workspace = workspace
    # End timing
    iteration_end = time.time()
    iteration_elapsed = int(iteration_end - iteration_start)
    iteration_success_time = datetime.datetime.now()
    # Report success
    print(f'Process will form composite from {len(tile_list)} raster tiles...')
    print(f'Raster list completed at {iteration_success_time.strftime("%Y-%m-%d %H:%M")} (Elapsed time: {datetime.timedelta(seconds=iteration_elapsed)})')
    print('----------')

    # Define projection and reproject all rasters in list
    print(f'Reprojecting {len(tile_rasters)} raster tiles...')
    # Define the initial projection
    tile_projection = arcpy.SpatialReference(input_projection)
    # Define the target projection
    composite_projection = arcpy.SpatialReference(output_projection)
    # Set snap raster
    arcpy.env.snapRaster = snap_raster
    # Set initial counter
    count = 1
    # Reproject all rasters in list
    for raster in tile_rasters:
        # Define intermediate and output raster
        reprojected_raster = os.path.join(projected_folder, os.path.splitext(os.path.split(raster)[1])[0] + '_reprojected.tif')
        output_raster = os.path.join(projected_folder, os.path.splitext(os.path.split(raster)[1])[0] + '.tif')
        # Check if output raster already exists:
        if os.path.exists(output_raster) == 0:
            # Start timing function
            iteration_start = time.time()
            print(f'\tReprojecting tile {count} of {len(tile_list)}...')
            # Define initial projection
            arcpy.DefineProjection_management(raster, tile_projection)
            # Reproject tile
            arcpy.ProjectRaster_management(raster,
                                           reprojected_raster,
                                           composite_projection,
                                           'BILINEAR',
                                           cell_size,
                                           geographic_transformation)
            # Enforce new projection
            arcpy.DefineProjection_management(reprojected_raster, composite_projection)
            # Set values less than -50 to null
            corrected_raster = SetNull(reprojected_raster, reprojected_raster, 'VALUE < -50')
            # Round to integer and store as 16 bit signed raster
            integer_raster = Int(Raster(corrected_raster) + 0.5)
            # Convert corrected raster to 16 bit signed
            arcpy.CopyRaster_management(integer_raster, output_raster, '', '', '-32768', 'NONE', 'NONE', '16_BIT_SIGNED','NONE', 'NONE', 'TIFF', 'NONE')
            # Delete intermediate raster
            arcpy.Delete_management(reprojected_raster)
            # End timing
            iteration_end = time.time()
            iteration_elapsed = int(iteration_end - iteration_start)
            iteration_success_time = datetime.datetime.now()
            # Report success for iteration
            print(f'\tFinished reprojecting tile {count} of {len(tile_list)}...')
            print(f'\tTile projection completed at {iteration_success_time.strftime("%Y-%m-%d %H:%M")} (Elapsed time: {datetime.timedelta(seconds=iteration_elapsed)})')
            print('\t----------')
        # Return message if output raster already exists
        else:
            print(f'\tTile {count} of {len(tile_list)} already processed...')
        # Increase count
        count += 1
    # Report success for loop
    print(f'Defined projection for {len(tile_list)} raster tiles...')
    print('----------')

    # Start timing function
    iteration_start = time.time()
    # Create a list of projected DEM raster tiles
    print('Compiling list of projected raster tiles...')
    arcpy.env.workspace = projected_folder
    projected_list = arcpy.ListRasters('*', 'ALL')
    # Add file path to raster list
    projected_rasters = []
    for tile in projected_list:
        tile_path = os.path.join(projected_folder, tile)
        projected_rasters.append(tile_path)
    # Set environment workspace
    arcpy.env.workspace = workspace
    # End timing
    iteration_end = time.time()
    iteration_elapsed = int(iteration_end - iteration_start)
    iteration_success_time = datetime.datetime.now()
    # Report success
    print(f'Process will form composite from {len(tile_list)} raster tiles...')
    print(f'Raster list completed at {iteration_success_time.strftime("%Y-%m-%d %H:%M")} (Elapsed time: {datetime.timedelta(seconds=iteration_elapsed)})')
    print('----------')

    # Start timing function
    iteration_start = time.time()
    print('Creating composite from tiles...')
    # Mosaic raster tiles to new raster
    arcpy.MosaicToNewRaster_management(projected_rasters,
                                       mosaic_location,
                                       mosaic_name,
                                       composite_projection,
                                       '16_BIT_SIGNED',
                                       cell_size,
                                       '1',
                                       'MAXIMUM',
                                       'FIRST')
    # Enforce correct projection
    arcpy.DefineProjection_management(dem_composite, composite_projection)
    # End timing
    iteration_end = time.time()
    iteration_elapsed = int(iteration_end - iteration_start)
    iteration_success_time = datetime.datetime.now()
    # Report success
    print(f'Raster composite completed at {iteration_success_time.strftime("%Y-%m-%d %H:%M")} (Elapsed time: {datetime.timedelta(seconds=iteration_elapsed)})')
    print('----------')

    # Delete intermediate dataset
    out_process = 'Successful creating composite DEM.'
    return out_process

# 5 Meter DEM Prep
## Download tiles for region of interest using csv exported from <a href='https://viewer.nationalmap.gov/basic/#/elevation'> National Map Viewer</a>

In [ ]:

# ---------------------------------------------------------------------------
# Download Files From CSV
# Author: Timm Nawrocki
# Last Updated: 2019-10-29
# Usage: Can be executed in an Anaconda Python 3.7 distribution or an ArcGIS Pro Python 3.6 distribution.
# Description: "Download Files From CSV" contacts a server to download a series of files specified in a csv table. The full path to the download must be specified in the table.
# ---------------------------------------------------------------------------

# # Define a function to download files from a csv
# def download_from_csv(input_table, url_column, destination):
#     """
#     Description: downloads set of files specified in a particular column of a csv table.
#     Inputs: input_table -- csv table containing rows for download items.
#             url_column -- title for column containing download urls.
#             destination -- folder to store download results.
#     Returned Value: Function returns status messages only. Downloaded data are stored on drive.
#     Preconditions: csv tables must be generated from web application tools or manually.
#     """

#     # Import packages
#     import datetime
#     import pandas as pd
#     import time
#     import urllib

#     # Import a csv file with the download urls for the Arctic DEM tiles
#     download_items = pd.read_csv(input_table)

#     # Initialize download count
#     n = len(download_items[url_column])
#     print(f'Beginning download of {n} tiles...')
#     count = 1

#     # Loop through urls in the downloadURL column and download
#     for url in download_items[url_column]:
#         target = os.path.join(destination, os.path.split(url)[1])
#         if os.path.exists(target) == 0:
#             try:
#                 # Start timing function
#                 iteration_start = time.time()
#                 print(f'\tDownloading {count} of {n} tiles...')
#                 # Download data
#                 filedata = urllib.request.urlopen(url)
#                 datatowrite = filedata.read()
#                 with open(target, 'wb') as file:
#                     file.write(datatowrite)
#                     file.close()
#                 # End timing
#                 iteration_end = time.time()
#                 iteration_elapsed = int(iteration_end - iteration_start)
#                 iteration_success_time = datetime.datetime.now()
#                 # Report success
#                 print(f'\tCompleted at {iteration_success_time.strftime("%Y-%m-%d %H:%M")} (Elapsed time: {datetime.timedelta(seconds=iteration_elapsed)})')
#                 print('\t----------')
#             except:
#                 print(f'\tTile {count} of {n} not available for download. Check url.')
#                 print('\t----------')
#         else:
#             print(f'\tTile {count} of {n} already exists...')
#             print('\t----------')
#         count += 1

#     print('Finished downloading tiles.')

# download_from_csv(input_table, url_column, directory)

### Unzip raster tiles to data folder

In [ ]:
# import zipfile
# os.chdir(directory)
# start = datetime.datetime.now()
# ext = ".zip"
# for item in os.listdir(directory): # loop through items in dir
#     if item.endswith(ext):

#         file_name = os.path.abspath(item) # get full path of files
#         zip_ref = zipfile.ZipFile(file_name) # create zipfile object
#         zip_ref.extractall(data_folder) # extract file to dir
#         zip_ref.close() # close file
#         #os.remove(file_name) # delete zipped file if required
#         print ('Unzipping.....', file_name)
#         print('')

# print ('Unzipping complete')
# stop = datetime.datetime.now()
# elapsed = stop - start
# print ('Time to complete = ',elapsed)

## Merge Raster Tiles

In [ ]:
# # Define input and output arrays
# merge_tiles_inputs = [snap_raster]
# merge_tiles_outputs = [usgs5m_composite]
#
# # Create key word arguments
# merge_tiles_kwargs = {'tile_folder': tile_folder,
#                      'projected_folder': projected_folder,
#                      'workspace': arcpy.env.workspace,
#                      'cell_size': 5,
#                      'input_projection': 3338,
#                      'output_projection': 3338,
#                      'geographic_transformation': '',
#                      'input_array': merge_tiles_inputs,
#                      'output_array': merge_tiles_outputs
#                      }
#
# # Merge source tiles
# comp_dem = arcpy_geoprocessing(merge_elevation_tiles, **merge_tiles_kwargs)


### Next Step: Input DEM for processing with TauDEM

